In [1]:
import cv2
import numpy as np
import os

# Empty callback for trackbars
def nothing(x):
    pass

# Apply selected filter based on type
def apply_filter(gray, ksize, filter_type):
    if filter_type == 0:  # Blur
        k = ksize if ksize % 2 == 1 else ksize + 1
        return cv2.GaussianBlur(gray, (k, k), 0)
    elif filter_type == 1:  # Sharpen
        kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        return cv2.filter2D(gray, -1, kernel)
    elif filter_type == 2:  # Sobel X
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=ksize)
        return cv2.convertScaleAbs(sobel)
    elif filter_type == 3:  # Sobel Y
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=ksize)
        return cv2.convertScaleAbs(sobel)
    elif filter_type == 4:  # Laplacian
        lap = cv2.Laplacian(gray, cv2.CV_64F, ksize=ksize)
        return cv2.convertScaleAbs(lap)
    else:
        return gray

# Static image processing
def static_image_mode():
    script_dir = os.path.dirname(os.path.abspath(__file__))
    img_path = os.path.join(script_dir, 'Datos', 'firewatch.jpg')
    img = cv2.imread(img_path)
    if img is None:
        print(f"Cannot load image at {img_path}")
        return
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply filters
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharpen = cv2.filter2D(gray, -1, kernel)
    sobelx = apply_filter(gray, 3, 2)
    sobely = apply_filter(gray, 3, 3)
    laplacian = apply_filter(gray, 3, 4)
    # Save results to directory
    result_dir = os.path.join(script_dir, 'Capturas')
    os.makedirs(result_dir, exist_ok=True)
    cv2.imwrite(os.path.join(result_dir, 'original.jpg'), img)
    cv2.imwrite(os.path.join(result_dir, 'grayscale.jpg'), gray)
    cv2.imwrite(os.path.join(result_dir, 'blur.jpg'), blur)
    cv2.imwrite(os.path.join(result_dir, 'sharpen.jpg'), sharpen)
    cv2.imwrite(os.path.join(result_dir, 'sobelx.jpg'), sobelx)
    cv2.imwrite(os.path.join(result_dir, 'sobely.jpg'), sobely)
    cv2.imwrite(os.path.join(result_dir, 'laplacian.jpg'), laplacian)
    print(f"Saved processed images to {result_dir}")

    # Display results
    cv2.imshow('Original', img)
    cv2.imshow('Grayscale', gray)
    cv2.imshow('Blur (5x5)', blur)
    cv2.imshow('Sharpen', sharpen)
    cv2.imshow('Sobel X', sobelx)
    cv2.imshow('Sobel Y', sobely)
    cv2.imshow('Laplacian', laplacian)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Webcam mode with live controls
def webcam_mode():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Cannot open webcam")
        return
    cv2.namedWindow('Controls', cv2.WINDOW_NORMAL)
    cv2.createTrackbar('Filter', 'Controls', 0, 4, nothing)
    cv2.createTrackbar('Kernel', 'Controls', 1, 31, nothing)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        ftype = cv2.getTrackbarPos('Filter', 'Controls')
        k = cv2.getTrackbarPos('Kernel', 'Controls')
        k = max(1, k)
        if k % 2 == 0:
            k += 1
        filtered = apply_filter(gray, k, ftype)

        cv2.imshow('Webcam - Gray', gray)
        cv2.imshow('Webcam - Filtered', filtered)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Entry point
if __name__ == '__main__':
    mode = input("Enter 'w' for webcam mode or anything else for static image mode: ")
    if mode.lower() == 'w':
        webcam_mode()
    else:
        static_image_mode()


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2564: error: (-27:Null pointer) NULL window: 'Controls' in function 'cvGetTrackbarPos'


: 